In [46]:
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

In [47]:
A = pd.read_csv('input/combos.csv')
C = pd.read_csv('input/menu_byob.csv')
P = pd.read_csv('input/menu_combo.csv')
order = pd.read_csv('input/cust_order.csv')
D = order

In [48]:
A = A.fillna(0)
A = A.set_index('combo')
A.sort_index(inplace=True)
A = A.reindex(sorted(A.columns), axis=1)

In [49]:
C = C.set_index('seafood')
C.sort_index(inplace=True)

In [50]:
P = P.set_index('combo')
P.sort_index(inplace=True)

In [51]:
# Total numbers of seafood types and combos

num_seafood = len(C)
num_combo = len(P)

seafoods = C['price'].keys().tolist()
seafoods[seafoods.index('ccm')] = 'crawfish-clams-mussels'
combos = P['price'].keys().tolist()

In [52]:
# "Customer order" info needs modifying on two places (1)(2)

In [53]:
# (1) Split number of "tails" into numbers of "tail_1" and "tail_2"

num_tails = D[D['seafood'] == 'tail']['pound'].values[0]
num_1_tail = num_tails % 2
num_2_tail = int(num_tails/2)

In [54]:
# (2) Crawfish, clams, and mussels have the same price and are interchangable
# Combine these items to form a new item, "ccm"

ccm_lbs = D[D['seafood'] == 'crawfish']['pound'].values[0] \
        + D[D['seafood'] == 'clams']['pound'].values[0] \
        + D[D['seafood'] == 'mussels']['pound'].values[0] 

In [55]:
# Add "df_add" to the D dataframe and delete "crawfish", "clams", "mussels", and "tail" from it.

dict_add = {'seafood':['ccm','tail_1','tail_2'], 
            'pound': [ccm_lbs, num_1_tail, num_2_tail]
           }

df_add = pd.DataFrame(dict_add)

D = pd.concat( [D, df_add], axis=0, ignore_index=True )

D.drop(D[D['seafood'] == 'crawfish'].index, inplace=True)
D.drop(D[D['seafood'] == 'clams'].index, inplace=True)
D.drop(D[D['seafood'] == 'mussels'].index, inplace=True)
D.drop(D[D['seafood'] == 'tail'].index, inplace=True)

In [56]:
# Cast all numeric info into dictionaries for modeling

comboMakeUp = A.values.tolist()
priceByob = C['price'].tolist()
priceCombo = P['price'].tolist()

D = D.set_index('seafood')
D.sort_index(inplace=True)
demandLBS = D['pound'].tolist()

In [57]:
# (OPTIONAL) Retrieve the index of 'tail_1' and 'tail_2'

D_temp = D.reset_index()
index_tail_1 = D_temp.index[D_temp['seafood'] == 'tail_1'].tolist()[0]
index_tail_2 = D_temp.index[D_temp['seafood'] == 'tail_2'].tolist()[0]

In [58]:
# BYOB Price of the order can be computed immediately

# totalByob = 0
# for i in range(num_seafood):
#     totalByob = totalByob + priceByob[i] * demandLBS[i]

totalByob = sum([u * v for u,v in zip(priceByob, demandLBS)])

In [59]:
# Create the MIP solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

infinity = solver.infinity()

# Define decision variables, x[i]: build your own bag seafood, y[j]: numbers of combos

x = {}
for i in range(num_seafood):
    x[i] = solver.IntVar(0, infinity, "x[%i]" % i)

y = {}
for j in range(num_combo):
    y[j] = solver.IntVar(0, infinity, "y[%i]" % j)
    
print("Number of variables: ", solver.NumVariables())

# Define constraints

## Constraint (1): customer demand must be met
for i in range(num_seafood):
    constraint_expr = 0      
    for j in range(num_combo):
        constraint_expr = constraint_expr + comboMakeUp[i][j] * y[j]         
    solver.Add( constraint_expr + x[i] >= demandLBS[i] )

## Constraint (2): lobster tails (optional)
# solver.Add( x[index_tail_2] == 2*x[index_tail_1] )  


# Define objective function

objective = solver.Objective()

obj_expr =  0
for j in range(num_combo):
    obj_expr = obj_expr + priceCombo[j] * y[j]
for i in range(num_seafood):
    obj_expr = obj_expr + priceByob[i] * x[i]

solver.Minimize(obj_expr)

# Call the solver

print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()

# Postprocessing

# Display the solution

if status == pywraplp.Solver.OPTIMAL:
    print()
    print(f"Problem solved in {solver.wall_time():d} milliseconds")
    print(f"Problem solved in {solver.iterations():d} iterations")
    print(f"Problem solved in {solver.nodes():d} branch-and-bound nodes")
    print( )
    print("=============== THE KING CRAB HACK ===============")
    print( )
    print("Here's everything you ordered: ")
    print( )
    print(order)
    print( )
    print("'Build Your Own Bag' would have cost: $", round(totalByob,2))
    print("\n" "Here's what you should order to get a bang for the buck:" "\n")
    for j in range(num_combo):
        if y[j].solution_value() != 0: 
            print(combos[j], " = ", int(y[j].solution_value()))
    for i in range(num_seafood):
        if x[i].solution_value() != 0: 
            print(seafoods[i], " = ", int(x[i].solution_value()))  
    print( )
    print("!! Now, your total (objective value) is: $", round(solver.Objective().Value(),2))
    print("!! YOU SAVED: $", round(totalByob-solver.Objective().Value(),2), "(%s)" % format((totalByob-solver.Objective().Value())/totalByob, ".0%") )
    
else:
    print("The problem does not have an optimal solution.")
    

Number of variables:  31
Solving with SCIP 8.1.0 [LP solver: Glop 9.9]

Problem solved in 9 milliseconds
Problem solved in 10 iterations
Problem solved in 1 branch-and-bound nodes

=============== THE KING CRAB HACK ===============

Here's everything you ordered: 

      seafood  pound
0    crawfish      3
1     mussels      1
2       clams      1
3        king      3
4        tail      0
5    lob_live      0
6    scallops      0
7   shrimp_wh      0
8   shrimp_hs      3
9        snow      3
10  dungeness      0
11       corn      5
12     potato      5
13    sausage      5

'Build Your Own Bag' would have cost: $ 411.31

Here's what you should order to get a bang for the buck:

combo_2  =  2
combo_4_hs  =  1
combo_sp2_hs  =  4
king  =  1

!! Now, your total (objective value) is: $ 372.92
!! YOU SAVED: $ 38.39 (9%)
